In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# import f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from tensorflow.keras import layers, regularizers  # Import regularizers here

#from datasets import load_dataset



In [2]:
all_train_df = pd.read_csv("datasets/train.csv").rename(columns={"prompt": "text", "type": "label"})
test_df = pd.read_csv("datasets/test.csv").rename(columns={"prompt": "text", "type": "label"})



In [3]:
# shuffle and with random seed for reproducibility and split the data into train and val 0.8/0.2
all_train_df = all_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training (80%) and validation (20%)
train_size = int(0.8 * len(all_train_df))
train_df = all_train_df[:train_size].reset_index(drop=True)
val_df = all_train_df[train_size:].reset_index(drop=True)

In [4]:


# Preprocess text data
max_vocab_size = 10000
max_sequence_length = 100

# Tokenization
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(train_df['text'])
sequences = tokenizer.texts_to_sequences(train_df['text'])
X_train = pad_sequences(sequences, maxlen=max_sequence_length)
X_val = pad_sequences(tokenizer.texts_to_sequences(val_df['text']), maxlen=max_sequence_length)

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_val = label_encoder.transform(val_df['label'])


# Define the model
model = tf.keras.Sequential([
    layers.Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    # Single output neuron for binary classification
    layers.Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

# Compile the model with binary crossentropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
y_test = label_encoder.transform(test_df['label'])



2024-11-05 15:49:42.663430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-05 15:49:42.663487: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2024-11-05 15:49:44.038923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-05 15:49:44.099705: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


27/27 [==============================] - ETA: 0s - loss: 0.6308 - accuracy: 0.7988

2024-11-05 15:49:48.614185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 5s 83ms/step - loss: 0.6308 - accuracy: 0.7988 - val_loss: 0.5464 - val_accuracy: 0.8038
Epoch 2/10
27/27 [==============================] - 1s 50ms/step - loss: 0.4553 - accuracy: 0.8192 - val_loss: 0.3900 - val_accuracy: 0.8134
Epoch 3/10
27/27 [==============================] - 1s 41ms/step - loss: 0.3057 - accuracy: 0.8575 - val_loss: 0.3070 - val_accuracy: 0.8660
Epoch 4/10
27/27 [==============================] - 1s 41ms/step - loss: 0.2051 - accuracy: 0.9401 - val_loss: 0.2366 - val_accuracy: 0.9282
Epoch 5/10
27/27 [==============================] - 1s 40ms/step - loss: 0.1366 - accuracy: 0.9641 - val_loss: 0.1983 - val_accuracy: 0.9426
Epoch 6/10
27/27 [==============================] - 1s 41ms/step - loss: 0.0958 - accuracy: 0.9760 - val_loss: 0.1816 - val_accuracy: 0.9378
Epoch 7/10
27/27 [==============================] - 1s 37ms/step - loss: 0.0717 - accuracy: 0.9832 - val_loss: 0.1721 - val_accuracy: 0.9426
Epoch 8/10
27/27 [======

In [5]:
y_probs = model.predict(X_test).ravel()
y_pred = (y_probs > 0.5).astype(int)
_f1_score = f1_score(y_test, y_pred)
_accuracy = accuracy_score(y_test, y_pred)
_recall = recall_score(y_test, y_pred)
_precision = precision_score(y_test, y_pred)
print(f"F1 Score: {_f1_score}")
print(f"Accuracy: {_accuracy}")
print(f"Recall: {_recall}")
print(f"Precision: {_precision}")

9/9 [==============================] - 0s 16ms/step


2024-11-05 15:49:58.953931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


F1 Score: 0.9781021897810218
Accuracy: 0.9770992366412213
Recall: 0.9640287769784173
Precision: 0.9925925925925926


In [6]:
def train_and_evaluate_model(embedding_dim, dense_units, dropout_rate):
    model = tf.keras.Sequential([
        layers.Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        layers.GlobalAveragePooling1D(),
        
        # First Dense Layer
        layers.Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        layers.Dropout(rate=dropout_rate),  # Dropout after first dense layer

        
        # Output layer for binary classification
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)
    
    return history

# Define different configurations to try
configs = [
    (128, 64, 0.5),
    (128, 64, 0.4),
    (256, 128, 0.4),
    (256, 64, 0.3),  # You can adjust these configurations as needed
    (128, 64, 0.1),
    (128, 64, 0.0),

]
best_params = {}
# Iterate through configurations
for embedding_dim, dense_units, dropout_rate in configs:
    print(f'Training model with embedding_dim={embedding_dim}, dense_units={dense_units}, dropout_rate={dropout_rate}')
    history = train_and_evaluate_model(embedding_dim, dense_units, dropout_rate)
    if not best_params or history.history['val_accuracy'][-1] > best_params['val_accuracy']:
        best_params = {
            'embedding_dim': embedding_dim,
            'dense_units': dense_units,
            'dropout_rate': dropout_rate,
            'val_accuracy': history.history['val_accuracy'][-1]
        }


Training model with embedding_dim=128, dense_units=64, dropout_rate=0.5
Epoch 1/10


2024-11-05 15:49:59.549699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-05 15:49:59.611075: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


27/27 [==============================] - ETA: 0s - loss: 0.7212 - accuracy: 0.7713

2024-11-05 15:50:01.894849: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 3s 78ms/step - loss: 0.7212 - accuracy: 0.7713 - val_loss: 0.6564 - val_accuracy: 0.7990
Epoch 2/10
27/27 [==============================] - 2s 62ms/step - loss: 0.5893 - accuracy: 0.8359 - val_loss: 0.5272 - val_accuracy: 0.8038
Epoch 3/10
27/27 [==============================] - 2s 57ms/step - loss: 0.4482 - accuracy: 0.8659 - val_loss: 0.4198 - val_accuracy: 0.8134
Epoch 4/10
27/27 [==============================] - 1s 49ms/step - loss: 0.3438 - accuracy: 0.9030 - val_loss: 0.3541 - val_accuracy: 0.8947
Epoch 5/10
27/27 [==============================] - 1s 50ms/step - loss: 0.2757 - accuracy: 0.9425 - val_loss: 0.3005 - val_accuracy: 0.9282
Epoch 6/10
27/27 [==============================] - 1s 50ms/step - loss: 0.2150 - accuracy: 0.9569 - val_loss: 0.2707 - val_accuracy: 0.9378
Epoch 7/10
27/27 [==============================] - 1s 50ms/step - loss: 0.1809 - accuracy: 0.9629 - val_loss: 0.2538 - val_accuracy: 0.9330
Epoch 8/10
27/27 [======

...

27/27 [==============================] - 4s 118ms/step - loss: 0.7477 - accuracy: 0.7928 - val_loss: 0.6136 - val_accuracy: 0.7990
Epoch 2/10
27/27 [==============================] - 2s 75ms/step - loss: 0.4998 - accuracy: 0.8192 - val_loss: 0.4471 - val_accuracy: 0.8134
Epoch 3/10
27/27 [==============================] - 2s 79ms/step - loss: 0.3405 - accuracy: 0.8982 - val_loss: 0.3433 - val_accuracy: 0.9139
Epoch 4/10
27/27 [==============================] - 2s 63ms/step - loss: 0.2375 - accuracy: 0.9557 - val_loss: 0.2871 - val_accuracy: 0.9378
Epoch 5/10
27/27 [==============================] - 2s 73ms/step - loss: 0.1872 - accuracy: 0.9725 - val_loss: 0.2584 - val_accuracy: 0.9426
Epoch 6/10
27/27 [==============================] - 2s 64ms/step - loss: 0.1550 - accuracy: 0.9772 - val_loss: 0.2374 - val_accuracy: 0.9426
Epoch 7/10
27/27 [==============================] - 2s 75ms/step - loss: 0.1305 - accuracy: 0.9820 - val_loss: 0.2286 - val_accuracy: 0.9378
Epoch 8/10
27/27 [=====

2024-11-05 15:50:51.466636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-05 15:50:51.530986: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


27/27 [==============================] - ETA: 0s - loss: 0.7105 - accuracy: 0.7772

2024-11-05 15:50:54.627533: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 4s 114ms/step - loss: 0.7105 - accuracy: 0.7772 - val_loss: 0.6043 - val_accuracy: 0.8038
Epoch 2/10
27/27 [==============================] - 2s 74ms/step - loss: 0.5052 - accuracy: 0.8275 - val_loss: 0.4427 - val_accuracy: 0.8134
Epoch 3/10
27/27 [==============================] - 2s 65ms/step - loss: 0.3620 - accuracy: 0.8766 - val_loss: 0.3636 - val_accuracy: 0.8612
Epoch 4/10
27/27 [==============================] - 2s 61ms/step - loss: 0.2585 - accuracy: 0.9425 - val_loss: 0.3013 - val_accuracy: 0.9330
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2032 - accuracy: 0.9665 - val_loss: 0.2655 - val_accuracy: 0.9426
Epoch 6/10
27/27 [==============================] - 2s 67ms/step - loss: 0.1688 - accuracy: 0.9713 - val_loss: 0.2488 - val_accuracy: 0.9426
Epoch 7/10
27/27 [==============================] - 2s 65ms/step - loss: 0.1434 - accuracy: 0.9820 - val_loss: 0.2329 - val_accuracy: 0.9330
Epoch 8/10
27/27 [=====

In [9]:
best_params

{'embedding_dim': 256,
 'dense_units': 128,
 'dropout_rate': 0.4,
 'val_accuracy': 0.9473684430122375}

In [10]:
X_train_all = pad_sequences(tokenizer.texts_to_sequences(all_train_df['text']), maxlen=max_sequence_length)
y_train_all = label_encoder.transform(all_train_df['label'])

In [11]:
best_model = tf.keras.Sequential([
    layers.Embedding(input_dim=max_vocab_size, output_dim=best_params['embedding_dim'], input_length=max_sequence_length),
    layers.GlobalAveragePooling1D(),
    layers.Dense(best_params['dense_units'], activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(rate=best_params['dropout_rate']),
    layers.Dense(1, activation='sigmoid')
])

best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = best_model.fit(X_train_all, y_train_all, epochs=10, validation_data=(X_val, y_val), batch_size=32)



Epoch 1/10


2024-11-05 15:53:42.458972: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-05 15:53:42.541551: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


33/33 [==============================] - ETA: 0s - loss: 0.7259 - accuracy: 0.7768

2024-11-05 15:53:46.767877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 5s 125ms/step - loss: 0.7259 - accuracy: 0.7768 - val_loss: 0.5652 - val_accuracy: 0.8038
Epoch 2/10
33/33 [==============================] - 3s 82ms/step - loss: 0.4546 - accuracy: 0.8266 - val_loss: 0.3694 - val_accuracy: 0.8804
Epoch 3/10
33/33 [==============================] - 2s 72ms/step - loss: 0.2960 - accuracy: 0.9272 - val_loss: 0.2517 - val_accuracy: 0.9569
Epoch 4/10
33/33 [==============================] - 2s 75ms/step - loss: 0.2117 - accuracy: 0.9598 - val_loss: 0.1964 - val_accuracy: 0.9617
Epoch 5/10
33/33 [==============================] - 2s 69ms/step - loss: 0.1639 - accuracy: 0.9741 - val_loss: 0.1618 - val_accuracy: 0.9617
Epoch 6/10
33/33 [==============================] - 2s 74ms/step - loss: 0.1376 - accuracy: 0.9780 - val_loss: 0.1380 - val_accuracy: 0.9713
Epoch 7/10
33/33 [==============================] - 2s 61ms/step - loss: 0.1196 - accuracy: 0.9818 - val_loss: 0.1228 - val_accuracy: 0.9713
Epoch 8/10
33/33 [=====

In [ ]:
y_probs = best_model.predict(X_test).ravel()
y_pred = (y_probs > 0.5).astype(int)
_f1_score = f1_score(y_test, y_pred)
_accuracy = accuracy_score(y_test, y_pred)
_recall = recall_score(y_test, y_pred)
_precision = precision_score(y_test, y_pred)
print(f"F1 Score: {_f1_score}")
print(f"Accuracy: {_accuracy}")
print(f"Recall: {_recall}")
print(f"Precision: {_precision}")